Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import rdflib
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
from rdflib.namespace import DC, FOAF

from owlready2 import *
from owlready2 import get_ontology

import networkx as nx
import networkx.algorithms.community as nx_comm

from sklearn.decomposition import PCA
import seaborn as sns

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


Loading the triples

In [2]:
# load csv medsur.csv
colnames = ["subject", "predicate", "object"]
triples_df = pd.read_csv('medsur.csv', names=colnames, header=None)

triples_df['object'] = triples_df['object'].str.rstrip()
triples_df['subject'] = triples_df['subject'].str.rstrip() 
triples_df['predicate'] = triples_df['predicate'].str.rstrip() 

triples_df = triples_df.drop_duplicates()
triples_df = triples_df.dropna()

# filter out all triples that contain /nan
triples_df = triples_df[~triples_df['subject'].str.contains("/nan")]
triples_df = triples_df[~triples_df['object'].str.contains("/nan")]

print(triples_df.head())

                               subject  \
0  http://www.medsur.org/patient_10507   
1  http://www.medsur.org/patient_10594   
2   http://www.medsur.org/patient_4479   
3  http://www.medsur.org/patient_13760   
4   http://www.medsur.org/patient_2136   

                                   predicate  \
0   http://example.org/medsur.rdf#hasOutcome   
1       http://example.org/medsur.rdf#hasLLT   
2  http://example.org/medsur.rdf#hasAgeGroup   
3      http://example.org/medsur.rdf#hasHLGT   
4       http://example.org/medsur.rdf#hasSOC   

                                   object  
0   http://www.medsur.org/outcome/Ongoing  
1  http://www.medsur.org/symptom/10046901  
2         http://www.medsur.org/age/25_44  
3     http://www.medsur.org/hlgt/10018424  
4      http://www.medsur.org/soc/10007541  


Filter triples to get symptoms

In [3]:
# only select triples that contains the predicate 'hasPT' and 'hasLLT'
triples_df = triples_df[(triples_df['predicate'] == 'http://example.org/medsur.rdf#hasPT') | (triples_df['predicate'] == 'http://example.org/medsur.rdf#hasLLT')]
print(triples_df.head())

                                subject                             predicate  \
1   http://www.medsur.org/patient_10594  http://example.org/medsur.rdf#hasLLT   
6   http://www.medsur.org/patient_12752   http://example.org/medsur.rdf#hasPT   
14    http://www.medsur.org/patient_418   http://example.org/medsur.rdf#hasPT   
17  http://www.medsur.org/patient_13840  http://example.org/medsur.rdf#hasLLT   
20    http://www.medsur.org/patient_988   http://example.org/medsur.rdf#hasPT   

                                    object  
1   http://www.medsur.org/symptom/10046901  
6   http://www.medsur.org/symptom/10016334  
14  http://www.medsur.org/symptom/10017577  
17  http://www.medsur.org/symptom/10021597  
20  http://www.medsur.org/symptom/10047290  


In [4]:
triples_df.predicate.unique() # looks correct

array(['http://example.org/medsur.rdf#hasLLT',
       'http://example.org/medsur.rdf#hasPT'], dtype=object)

Create entities

In [7]:
# create np array of triples [[row1], [row2], ...]
triples = triples_df.values
triples[0] # looks correct

array(['http://www.medsur.org/patient_10594',
       'http://example.org/medsur.rdf#hasLLT',
       'http://www.medsur.org/symptom/10046901'], dtype=object)

Train-test split

In [9]:
from ampligraph.evaluation import train_test_split_no_unseen

test_size = int(0.1*len(triples_df))

X_train, X_test = train_test_split_no_unseen(triples, test_size=test_size)

print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (26370, 3)
Test set size:  (2930, 3)


We choose ComplEx as our Knowledge Graph Embedding model:

In [10]:
from ampligraph.latent_features.models import ScoringBasedEmbeddingModel
    
# Initialize a ComplEx neural embedding model: the embedding size is k,
# eta specifies the number of corruptions to generate per each positive,
# scoring_type determines the scoring function of the embedding model.
model = ScoringBasedEmbeddingModel(k=150,
                                   eta=10,
                                   scoring_type='ComplEx')

Compile model

In [11]:
import tensorflow as tf

from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer

# Optimizer, loss and regularizer definition
optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = get_loss('pairwise', {'margin': 0.5})
regularizer = get_regularizer('LP', {'p': 2, 'lambda': 1e-5})

# Compilation of the model
model.compile(loss=loss,
              optimizer='adam',
              entity_relation_regularizer=regularizer,
              entity_relation_initializer='glorot_uniform')

Train model

In [ ]:
# Fit the model on training and validation set

history = model.fit(X_train,
          batch_size=64, # use 1/10 of the training set as batch size
          epochs=100,                    # Number of training epochs
          verbose=True                  # Enable stdout messages
          )

An additional step when evaluating KGEs: Define a filter so that no negative statements generated by the corruption procedure are actually positives.

In [ ]:
filter = {'test': np.concatenate([X_train, X_test])}

Evaluate model

In [ ]:
ranks = model.evaluate(X_test,
                       use_filter=filter,
                       corrupt_side='s,o',
                       verbose=True)

We used the mrr_score (mean reciprocal rank) and hits_at_n_score functions to evaluate the quality of our predictions:

In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

Get embeddings for PTCodes

In [ ]:
symptoms = triples_df.object.unique()
embeddings = dict(zip(symptoms, model.get_embeddings(symptoms)))

We use PCA to project the embeddings from the 200 space into 2D space:

In [ ]:
embeddings_2d = PCA(n_components=2).fit_transform(np.array([i for i in embeddings.values()]))

Find the ideal k-value

In [ ]:
from sklearn.cluster import KMeans

WCSS = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, n_init=50, max_iter=500, random_state=0,init = 'k-means++')
    km.fit(embeddings_2d )
    WCSS.append(km.inertia_)

# plot
plt.plot(range(1, 11), WCSS, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show() # Based on elbow method, k should be 3.

We now cluster the teams embeddings on its original 200-dimensional space:

In [ ]:
from ampligraph.discovery import find_clusters

clustering_algorithm = KMeans(n_clusters=3, n_init=50, max_iter=500, random_state=0)
clusters = find_clusters(symptoms, model, clustering_algorithm, mode='e')

print(len(clusters))
print(len(symptoms))

Store the clusters for later use

In [ ]:
# Cluster-patient dictionary

results = dict(zip(clusters, symptoms))

with open("clusters_entities.txt", 'w') as f: 
    for key, value in results.items(): 
        f.write('%s:%s\n' % (key, value))

Visualization

In [ ]:
plot_df = pd.DataFrame({"symptoms": symptoms, 
                        "embedding1": embeddings_2d[:, 0], 
                        "embedding2": embeddings_2d[:, 1],
                        "cluster": "cluster" + pd.Series(clusters).astype(str)})

In [ ]:
# Plot 2D embeddings
def plot_clusters(hue):
    plt.figure(figsize=(12, 12))
    plt.title("{} embeddings".format(hue).capitalize())
    ax = sns.scatterplot(data=plot_df,
                         x="embedding1", y="embedding2", hue=hue)

In [ ]:
plot_clusters("cluster")